## Webscraping Definitions Using a List of Words with Part of Speech Tags

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

### Learn How to Use a WebDriver

In [ ]:
url = "https://www.oxfordlearnersdictionaries.com/search/english/?q=hello"

In [ ]:
driver = webdriver.Chrome()
driver.minimize_window()
driver.get(url)

### Learn How to Use BeautifulSoup

In [ ]:
content = driver.page_source
soup = BeautifulSoup(content)

In [ ]:
pos = soup.find_all("a", {"class": "selected"})

In [ ]:
pos

In [ ]:
definitions = soup.find_all("span", {"class": "def"})

In [ ]:
len(definitions)

In [ ]:
definitions[0]

In [ ]:
pattern = r'<[^>]+>'

In [ ]:
>>> re.sub(pattern, '', str(definitions[0]))

In [ ]:
>>> re.sub('<[^>]+>', '', str(pos))

### Read the File with a List of Words

In [ ]:
df = pd.read_csv("C1_tags.csv")

In [ ]:
df

In [ ]:
df[df['english'] == 'downward']

In [ ]:
### Look for n/a values
for i in range(len(df)):
    if type(df.loc[i]['english']) == float:
        print(i)

In [ ]:
df_definitions = []

In [ ]:
#Catch first 5 definitions:
for i in range(0, 5):
    word = df.loc[i]['english']
    word = word.lower()
    tag = df.loc[i]['tag']
    print(word)
    print(tag)
    driver = webdriver.Chrome()
    driver.get("https://www.oxfordlearnersdictionaries.com/definition/english/"+word)
    time.sleep(5)
    content = driver.page_source
    soup = BeautifulSoup(content)
    #print(soup)
    pos = soup.find_all("a", {"class": "selected"})
    print("PRINTING POS")
    #print(pos)
    pattern = r'<[^>]+>'
    cleaned_pos = re.sub('<[^>]+>', '', str(pos))
    print(cleaned_pos)
    definitions = soup.find_all("span", {"class": "def"})
    for i in range(len(definitions)):
        cleaned_def = re.sub(pattern, '', str(definitions[i]))
        definitions[i] = cleaned_def
        #print(cleaned_def)
    #print("CLEANED DEFS:")
    #print(definitions)
    if str(tag) == 'nan':
        if(len(definitions) > 0):
            df_definitions.append(definitions[0])
        else:
            df_definitions.append('NOTFOUND')
    elif 'noun' not in cleaned_pos and 'verb' not in cleaned_pos and 'adverb' not in cleaned_pos and 'exclamation' not in cleaned_pos and 'adjective' not in cleaned_pos and 'preposition' not in cleaned_pos and 'conjunction' not in cleaned_pos:
        if(len(definitions) > 0):
            df_definitions.append(definitions[0])
        else:
            df_definitions.append('NOTFOUND')
    else:
        c = 2
        loop_definitions = ''
        while tag not in cleaned_pos:
            if c > 4:
                c = 10
                break
            time.sleep(5)
            driver = webdriver.Chrome()
            driver.get("https://www.oxfordlearnersdictionaries.com/definition/english/"+word+"_"+str(c))
            content = driver.page_source
            soup = BeautifulSoup(content)
            pos = soup.find_all("a", {"class": "selected"})
            loop_definitions = soup.find_all("span", {"class": "def"})
            pattern = r'<[^>]+>'
            cleaned_pos = re.sub(pattern, '', str(pos))
            c = c+1
            
        if c != 10:
            loop_definitions = soup.find_all("span", {"class": "def"})
            definitions = loop_definitions
            for i in range(len(definitions)):
                cleaned_def = re.sub(pattern, '', str(definitions[i]))
                definitions[i] = cleaned_def
        #print("PRINTING DEFS:")
        #print(definitions)
        def_string = ''
        if len(definitions)> 1:
            for i in range(len(definitions)):
                def_string = def_string + str(i+1) + ". " + definitions[i]
                if i < len(definitions)-1:
                    def_string = def_string + " "
                    
            df_definitions.append(def_string)
        else:
            if(len(definitions) > 0):
                df_definitions.append(definitions[0])
            else:
                df_definitions.append('NOTFOUND')
        
                

In [ ]:
len(df_definitions)

In [ ]:
# Save the definitions to a txt file
with open('definitions_c1.txt', 'w', encoding='utf-8-sig') as f:
    for line in df_definitions:
        f.write("%s\n" % line)

In [ ]:
# !Only after all definitions are webscraped
# df['definition'] = df_definitions
# df.to_csv("c1_final.csv", index=False, encoding='utf-8-sig')

#### If list of definitions is too long you can cut leaving only first 3 definitions

In [ ]:
df = pd.read_csv("c1_final.csv")

In [ ]:
df

In [ ]:
definitions = df['definition'].tolist()

In [ ]:
definitions[111]

In [ ]:
for i in range(len(definitions)):
    if '4.' in definitions[i] :
        index = [m.start() for m in re.finditer('3.', definitions[i])]
        definitions[i] = definitions[i][:index[0]-1]

In [ ]:
df['definition'] = definitions

In [ ]:
# You might need to append some definitions manually
df[(df['definition'] == 'NOTFOUND') & (df['tag'].isna())]

In [ ]:
df['definition'] = definitions

In [ ]:
df

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.to_csv("c1_short.csv", index=False, encoding='utf-8-sig')